In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import scipy.stats as stats
import scipy.optimize as spo
from scipy.optimize import minimize

In [15]:
# standard deviation of spherical Gaussian distribution
mu = 0.0 
sigma_u = 1.0
sigma_v = 1.0
sigma = 0.2#[0, 0.2, 0.4, 0.6, 0.8, 1] 

D = 2 #dimension
N = 10 #nbr of users = 10
M = 10 #nbr of movies : M=5*alpha M = 10, 30, 100, 300, 1000
alpha_vals = [1, 3, 10, 30, 100] #[1,5,10,50,100]
k_best = 10 # 
nb_iter = 100 #1000

In [16]:
#generate matrices U, V and R from Gauss. distributions
# put all vars as arguments !!!!
def generate_U_V_R(mu, sigma_u, sigma_v, sigma, alpha):
    U = np.random.normal(mu, sigma_u, size=(D,N))
    V = np.random.normal(mu, sigma_v, size=(D,M*alpha))
    noise = np.random.normal(mu, sigma, size=(N,M*alpha)) 
    R = np.matmul(U.T,V) +noise
    return U, V, R

In [17]:
# define the objective function to optimize
def objective_function(V, R, U, sigma, alpha):
    V = V.reshape((D,M*alpha))
    lambda_U = sigma/sigma_u
    lambda_V = sigma/sigma_v
    error = R - np.dot(U.T, V)
    regularization_term = lambda_U/2 * np.linalg.norm(U)**2 + lambda_V/2 * np.linalg.norm(V)**2
    return  np.sum(error**2)*1/2 + regularization_term

In [20]:
#optimize the objective function 

def optimization_fn(alpha):
    M_alpha = M*alpha
    U, V, R = generate_U_V_R(mu, sigma_u, sigma_v, sigma, alpha)
    V0 = np.random.randn(D*M_alpha) # random guess for V0
    result = minimize(objective_function, V0, args=(R, U, sigma, alpha))# , options={"disp": True}
    V_result = 0
    if result.success :
        V_result = result.x.reshape((D, M_alpha))
    else :
        print('No minimum found')
    # frob_V = np.linalg.norm(V, 'fro')
    # frob_V_result = np.linalg.norm(V_result, 'fro')
    diff_norm = np.linalg.norm(V - V_result, 'fro')/np.sqrt(M_alpha)
    #print(f'Difference of Frobenius norm of V and V_result: {diff_norm}')
    return diff_norm


In [21]:
#compute average Difference of V and V_result for different values of M
for i in range(len(alpha_vals)):
    avg = 0
    for j in range(nb_iter):
        avg+= optimization_fn(alpha_vals[i])
    avg = avg/nb_iter
    print(f'Average Difference of Frobenius norm of V and V_result for M= {M*alpha_vals[i]}: {avg}')


Average Difference of Frobenius norm of V and V_result for M= 10: 0.11200898280429086
Average Difference of Frobenius norm of V and V_result for M= 30: 0.10685163697217571
Average Difference of Frobenius norm of V and V_result for M= 100: 0.11337903773253157
Average Difference of Frobenius norm of V and V_result for M= 300: 0.11069797194606645
Average Difference of Frobenius norm of V and V_result for M= 1000: 0.10976333512308084
